## Preface
The reason I created this notebook is to update the way to build the ONNX model of this repo, more specifically, the way the author builds the ONNX model here makes the ONNX model (FaceDetector.onnx) look like it supports dynamic batch size but in fact it is static batch size and even fixed at 1x3x1x1 which makes me encounter many difficulties and almost unable to convert .onnx to TensorRT engine as I want to use for my project. In this notebook, I will update the convert_to_onnx.py file to build a model that supports dynamic batch size. This helps the above errors and helps me to use the model in DeepStream with batch_size > 1 (multiple sources).

## Install dependencies


In [1]:
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 94.9 MB/s eta 0:00:00


In [2]:
import torch
import torchvision

In [3]:
print(torch.cuda.is_available())

True


In [4]:
import sys
import importlib

## Load files

Upload the original repository .zip file and unzip it, remember to rename the file if needed.

In [ ]:
!unzip RetinaFace-master.zip

Archive:  RetinaFace-master.zip
9534698bcfc7f77fedae600b22e7f6bf0616a31e
   creating: RetinaFace-master/
  inflating: RetinaFace-master/.gitattributes  
  inflating: RetinaFace-master/FaceDetector.onnx  
  inflating: RetinaFace-master/LICENSE.MIT  
  inflating: RetinaFace-master/README.md  
  inflating: RetinaFace-master/convert_to_onnx.py  
   creating: RetinaFace-master/curve/
  inflating: RetinaFace-master/curve/1.jpg  
  inflating: RetinaFace-master/curve/FDDB.png  
  inflating: RetinaFace-master/curve/Widerface.jpg  
  inflating: RetinaFace-master/curve/test.jpg  
   creating: RetinaFace-master/data/
   creating: RetinaFace-master/data/FDDB/
  inflating: RetinaFace-master/data/FDDB/img_list.txt  
  inflating: RetinaFace-master/data/__init__.py  
  inflating: RetinaFace-master/data/config.py  
  inflating: RetinaFace-master/data/data_augment.py  
  inflating: RetinaFace-master/data/wider_face.py  
  inflating: RetinaFace-master/detect.py  
  inflating: RetinaFace-master/detect_onnx

This is the pretrained/ weight folder that you can download on GG Drive.

In [ ]:
!unzip Retinaface_model_v2-20251103T073704Z-1-001.zip

Archive:  Retinaface_model_v2-20251103T073704Z-1-001.zip
  inflating: Retinaface_model_v2/mobilenet0.25_Final.pth  
  inflating: Retinaface_model_v2/mobilenetV1X0.25_pretrain.tar  
  inflating: Retinaface_model_v2/Resnet50_Final.pth  


## Setting up to rebuild ONNX model

Apply following changes on convert_to_onnx.py to rebuild ONNX

First, these are some changes which are required to make the file able to run on Google Colab.

In [43]:
# Add 'RetinaFace-master' to sys.path if not already present.
# This is necessary for Python to find 'models.retinaface' directly in this cell's context.
project_root = 'RetinaFace-master'
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Attempt to get the module; if it exists, reload it.
# This ensures the interpreter uses the updated file content.
module_name = 'models.retinaface'
if module_name in sys.modules:
    importlib.reload(sys.modules[module_name])

In [44]:
with open('RetinaFace-master/models/retinaface.py', 'r') as f:
  content = f.read()

# Replace the incorrect relative path with the correct one, relative to the current working directory
# The traceback indicates the file currently contains './weights/mobilenetV1X0.25_pretrain.tar'
new_content = content.replace(
    "./weights/mobilenetV1X0.25_pretrain.tar",
    "RetinaFace-master/weights/mobilenetV1X0.25_pretrain.tar"
)

with open('RetinaFace-master/models/retinaface.py', 'w') as f:
  f.write(new_content)

In [45]:
with open('RetinaFace-master/convert_to_onnx.py', 'r') as f:
  content = f.read()

# Replace the incorrect default path for --trained_model
new_content = content.replace(
    "default='./weights/mobilenet0.25_Final.pth'",
    "default='RetinaFace-master/weights/mobilenet0.25_Final.pth'"
)

with open('RetinaFace-master/convert_to_onnx.py', 'w') as f:
  f.write(new_content)

Second, apply this to build new ONNX model, here I opened the file on local machine and modify directly then upload here.

In [ ]:
# This one (apply it into the existing file)
# dynamic_axes = {"input": {0: "batch_size", 2: "height", 3: "width"}, "bbox": {1: "batch_size"}, "confidence": {1: "batch_size"}, "landmark": {1: "batch_size"}}

Overall, I did:
+ Modified the code to make it be able to run in Google Colab
+ Changed the line "dynamic_axes" (most important)
→ The old version looks dynamic, but it isn’t.
  Why:
  + "None" is not a valid label for dynamic axes.

  + PyTorch’s torch.onnx.export() only recognizes integer-to-string mapping, where the string is just a name label, e.g. "batch_size", "height", "width".
  + It doesn’t accept "None" as a special keyword — so effectively, this dictionary is ignored.
+ Used new name for new model.
+ Create new ONNX file with new name

Cat the file to see what I had changed

In [33]:
# Print the updated content to verify
with open('RetinaFace-master/convert_to_onnx.py', 'r') as f:
  print(f.read())

from __future__ import print_function
import os
import argparse
import torch
import torch.backends.cudnn as cudnn
import numpy as np
from data import cfg_mnet, cfg_re50
from layers.functions.prior_box import PriorBox
from utils.nms.py_cpu_nms import py_cpu_nms
import cv2
from models.retinaface import RetinaFace
from utils.box_utils import decode, decode_landm
from utils.timer import Timer


parser = argparse.ArgumentParser(description='Test')
parser.add_argument('-m', '--trained_model', default='RetinaFace-master/weights/mobilenet0.25_Final.pth',
                    type=str, help='Trained state_dict file path to open')
parser.add_argument('--network', default='mobile0.25', help='Backbone network mobile0.25 or resnet50')
parser.add_argument('--long_side', type=int, default=640, help='when origin_size is false, long_side is scaled size(320 or 640 for long side)')
parser.add_argument('--cpu', action="store_true", default=False, help='Use cpu inference')

args = parser.parse_args()


def 

## Rebuild command

In [8]:
%run RetinaFace-master/convert_to_onnx.py --trained_model RetinaFace-master/weights/Resnet50_Final.pth --network resnet50 --long_side 640

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 232MB/s]


load_to_cpu False
Loading pretrained model from RetinaFace-master/weights/Resnet50_Final.pth
remove prefix 'module.'
Missing keys:0
Unused checkpoint keys:0
Used keys:456
Finished loading model!
RetinaFace(
  (body): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_siz

/content/RetinaFace-master/convert_to_onnx.py:88: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch_out = torch.onnx.export(net, inputs, output_onnx, export_params=True, verbose=False,


Double check the code if needed.

In [ ]:
# # Clean up sys.path if we added it.
# if project_root in sys.path:
#     sys.path.remove(project_root)